# Lab 4 - MapReduce

In this lab, we are practicing the MapReduce programming paradigm. 

We will complete the tasks using the accompanied *mapreduce* package (as **mapreduce.py**) and MRJob. Please download the **mapreduce.py** file from our online class resource page, and place it in the same folder with your notebook.

Please also install MRJob through **pip install mrjob**.

For each invocation of an MapREduce job (with mr.run()), you are expected to supply a mapper, a reducer and/or a combiner as needed. Below are sample usage of the package:

```python
    # Run on input1 using your mapper1 and reducer1 function
    output = list(mr.run(input1, mapper1, reducer1))

    # Run on input2 using only your mapper2, no reduce phase
    output = list(mr.run(enumerate(input2), mapper2, combiner2))
    
    # Run on input3 using 2 nested MapReduce jobs
    output = mr.run(mr.run(input3, mapper3, reducer3), mapper4)
```
    
Please note that the input must be an iteratable of **key/value pairs**. If your inpu tdata does not have a key, you can simply add a null or index key through **enumerator(input)**. The output of the mr.run() is always a **generator**. You have to cast it to a list if you'd like to view, index or print it out.

The tasks below also include those that are in Homework 2, but we're using MapReduce instead of Python's general Higher Order Functions.

You will need **book.txt** and **citibike.csv** file from the class resource page.

In [9]:
import csv
import mapreduce as mr

## Task 0

Here is another concrete example on "Word Count" using the package. Assuming we have a text file named *book.txt*. Our task is to count the frequency of words in this document, and print the top 10. For illustration purposes, we use only the first 1000 lines of the book for counting.

In [19]:
with open('book.txt', 'r') as fi:
    lines = [(i,line.strip()) for i,line in enumerate(fi) if i<1000 ]

def mapper(_,line):
     for word in line.strip().split(' '):
            if len(word):
                yield (word,1)
def reducer(word,count):
    yield(word,sum(count))
output = list(mr.run(lines, mapper,reducer))
sorted_output=sorted(output,key = lambda x:-x[1])
sorted_output[:10]

[('the', 360),
 ('of', 326),
 ('and', 246),
 ('a', 169),
 ('or', 161),
 ('to', 101),
 ('with', 100),
 ('in', 88),
 ('on', 67),
 ('as', 56)]

### From Task 1 to Task 4, we re-do some tasks from Lab 3 (HOF) to familiarize ourselves with MapReduce

## Task 1

We would like to write a MapReduce job to count the total number of trips involved at each station. For example, if a trip starts at station A and stops at station B, the trip will count for both A and B. The output must be tuples, each consisting of a station name and a count.

In [36]:
def mapper1(_, row):
    yield(row['start_station_name'],1)
    yield(row['end_station_name'],1)

def reducer1(station, counts):
    yield(station,sum(counts))
    
with open('citibike.csv', 'r') as fi:
    reader = enumerate(csv.DictReader(fi))
    output1 = list(mr.run(reader, mapper1,reducer1))
output1[:10]

[('1 Ave & E 15 St', 795),
 ('1 Ave & E 44 St', 219),
 ('10 Ave & W 28 St', 422),
 ('11 Ave & W 27 St', 354),
 ('11 Ave & W 41 St', 461),
 ('11 Ave & W 59 St', 242),
 ('12 Ave & W 40 St', 217),
 ('2 Ave & E 31 St', 588),
 ('2 Ave & E 58 St', 125),
 ('3 Ave & Schermerhorn St', 34)]


## Task 2

Below is an example of showing how to use nested jobs and jobs with mappers only using the mapreduce package, thus, no points are included. Our task here is that we would like to filter the output of Task 1 to display only those stations with more than 1000 trips involved, of course, using the MapReduce paradigm.

In [39]:
def mapper2(station, count):
    if count>1000:
        yield(station,count)

with open('citibike.csv', 'r') as fi:
    reader = enumerate(csv.DictReader(fi))
    output2 = list(mr.run(mr.run(reader, mapper1, reducer1), mapper2))

output2

[('8 Ave & W 31 St', 1065),
 ('E 43 St & Vanderbilt Ave', 1003),
 ('Lafayette St & E 8 St', 1013),
 ('W 21 St & 6 Ave', 1057),
 ('W 41 St & 8 Ave', 1095)]


## Task 3

We would like to count the number of trips taken between pairs of stations. Trips taken from station A to station B or  from station B to station A are both counted towards the station pair A and B. Please note that the station pair shoud be identified by station names, as a tuple, and in lexical order, i.e. (A,B) instead of (B,A) in this case. The output must be tuples, each consisting of the station pair identification and a count.

In [42]:
def mapper3(_, row):
    station =  sorted([row['start_station_name'],row['end_station_name']])
    yield (station,1)
def reducer3(station_pair, counts):
    yield(station_pair, sum(counts))
    
with open('citibike.csv', 'r') as fi:
    reader = enumerate(csv.DictReader(fi))
    output3 = list(mr.run(reader, mapper3, reducer3))

output3[:10]

[(['1 Ave & E 15 St', '1 Ave & E 15 St'], 5),
 (['1 Ave & E 15 St', '1 Ave & E 44 St'], 6),
 (['1 Ave & E 15 St', '11 Ave & W 27 St'], 1),
 (['1 Ave & E 15 St', '2 Ave & E 31 St'], 9),
 (['1 Ave & E 15 St', '5 Ave & E 29 St'], 2),
 (['1 Ave & E 15 St', '6 Ave & Broome St'], 3),
 (['1 Ave & E 15 St', '6 Ave & Canal St'], 1),
 (['1 Ave & E 15 St', '8 Ave & W 31 St'], 5),
 (['1 Ave & E 15 St', '9 Ave & W 14 St'], 3),
 (['1 Ave & E 15 St', '9 Ave & W 16 St'], 3)]


## Task 4

In this task, you are asked to compute the station with the most riders started from, per each gender of the *'Subscriber'* user. Meaning, what was the station name with the highest number of bike pickups for female riders, for male riders and for unknown riders.

The output will be a list of tuples, each includes 
a gender label (as indicated below) and another tuple consisting of a station name, and the total number of trips started at that station for that gender.

The label mapping for the gender column in citibike.csv is: (Zero=<b>Unknown</b>; 1=<b>Male</b>; 2=<b>Female</b>)

In [77]:
def mapper4(_,row):
    if row['usertype']=='Subscriber':
        yield (row['gender'],row['start_station_name']),1
    
def reducer4(gender_station, counts):
    yield(gender_station,sum(counts))
    
def mapper5(gender_station,count):
    gender,station = gender_station
    yield(gender,(station,count))
    
def reducer5(gender,station_counts):
    max_station_count = max(station_counts,key=lambda x:x[1])
    label=('Unknown','Male','Female')
    yield (label[int(gender)], max_station_count)
    
with open('citibike.csv', 'r') as fi:
    reader = enumerate(csv.DictReader(fi))
    output5=list(mr.run(mr.run(reader,mapper4,reducer4),mapper5,reducer5))
    
output5

[('Unknown', ('Catherine St & Monroe St', 1)),
 ('Male', ('8 Ave & W 31 St', 488)),
 ('Female', ('W 21 St & 6 Ave', 107))]


## Task 5

We're going to tackle Task 3 of Homework 2 (or simply Homework 1) MapReduce.

In [101]:
def mapper6(_,row):
    yield((row['Customer ID'],row['Product ID']),float(row['Item Cost']))
    
def reducer6(pid_cid,costs):
    cid,pid = pid_cid
    yield(pid,sum(costs))
    
def mapper7(pid,cost):
    yield pid,cost
def reducer7(pid,costs):
    yield pid, round(sum(costs),2)
with open('sale.csv', 'r') as fi:
    reader = enumerate(csv.DictReader(fi))
    output3 = list(mr.run(mr.run(reader,mapper6,reducer6),mapper7,reducer7))

output3


[('P02291', 1181.97),
 ('P19498', 989.99),
 ('P32565', 1006.09),
 ('P33162', 1210.92),
 ('P39328', 1129.01),
 ('P58225', 1349.82),
 ('P61235', 959.02),
 ('P76615', 1087.96),
 ('P82222', 950.05),
 ('P92449', 966.17)]


## Task 6

MRJob is a convenient packages for simplifying the execution of MapReduce jobs on clusters. However, it doesn't work in a notebook. We're going to convert some of the examples of MRJob into our notebooks so that we can test our code before deploying them on Hadoop.

The two examples are available at:
https://pythonhosted.org/mrjob/guides/quickstart.html
https://pythonhosted.org/mrjob/guides/writing-mrjobs.html

[('the', 419),
 ('of', 343),
 ('and', 260),
 ('a', 196),
 ('or', 163),
 ('to', 104),
 ('with', 103),
 ('in', 90),
 ('coins', 71),
 ('on', 69)]

[(419, 'the')]


## Task 7

Let's try to run the above MRJob examples as stand-alone applications. Please check again:
https://pythonhosted.org/mrjob/guides/quickstart.html